# Retail Store Sales Optimization

**Goal**: Find optimal interventions to increase store sales by 20%

This notebook demonstrates how to use the Intervention Search system to identify the best ways to improve retail store performance through causal interventions.

## 1. Load Data and Setup

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import warnings
warnings.filterwarnings('ignore')

# Load retail data
df = pd.read_csv('data/retail_data.csv')
print(f"Loaded {len(df)} retail stores")
print(f"\nColumns: {list(df.columns)}")
df.head()

## 2. Define Causal Graph

**Causal Structure:**
- `store_location → foot_traffic → sales`
- `store_size → inventory_level → sales`
- `marketing_spend → foot_traffic`
- `price_discount → conversion_rate → sales`
- `staff_count → customer_satisfaction → sales`
- `competitor_proximity → foot_traffic`

In [ ]:
# Define causal graph as adjacency matrix
nodes = ['store_location', 'store_size', 'marketing_spend', 'price_discount', 
         'staff_count', 'competitor_proximity', 'foot_traffic', 'inventory_level', 
         'conversion_rate', 'customer_satisfaction', 'sales']

edges = [
    ('store_location', 'foot_traffic'),
    ('marketing_spend', 'foot_traffic'),
    ('competitor_proximity', 'foot_traffic'),
    ('store_size', 'inventory_level'),
    ('price_discount', 'conversion_rate'),
    ('staff_count', 'customer_satisfaction'),
    ('foot_traffic', 'sales'),
    ('inventory_level', 'sales'),
    ('conversion_rate', 'sales'),
    ('customer_satisfaction', 'sales')
]

# Create adjacency matrix
adj_matrix = pd.DataFrame(0, index=nodes, columns=nodes)
for parent, child in edges:
    adj_matrix.loc[parent, child] = 1

print("Causal Graph:")
print(adj_matrix)

## 3. Train Causal Model

In [ ]:
from ht_categ import HT, HTConfig

# Create and train HT model
config = HTConfig(graph=adj_matrix, model_type='XGBoost')
ht_model = HT(config)
ht_model.train(df)

print("✓ Causal model trained")
print(f"\nModel metrics (R² scores):")
for node, metrics in ht_model.model_metrics.items():
    if 'r2' in metrics:
        print(f"  {node}: {metrics['r2']:.3f}")

## 4. Find Optimal Interventions

**Objective**: Increase sales by 20% with high confidence

In [ ]:
from intervention_search import InterventionSearch

# Initialize intervention search
searcher = InterventionSearch(
    graph=ht_model.graph,
    ht_model=ht_model,
    n_simulations=1000
)

# Search for interventions to increase sales by 20%
results = searcher.find_interventions(
    target_outcome='sales',
    target_change=20.0,  # +20% increase
    tolerance=3.0,       # ±3% tolerance
    confidence_level=0.90,
    max_intervention_pct=30.0,
    verbose=True
)

## 5. Analyze Best Intervention

In [ ]:
best = results['best_intervention']

print("\n" + "="*70)
print("RECOMMENDED INTERVENTION")
print("="*70)
print(f"\nIntervene on: {', '.join(best['nodes'])}")
print(f"\nRequired changes:")
for node, change in best['required_pct_changes'].items():
    baseline = ht_model.baseline_stats[node]['mean']
    new_value = baseline * (1 + change/100)
    print(f"  • {node}: {change:+.1f}% (from {baseline:.0f} to {new_value:.0f})")

print(f"\nExpected Impact:")
print(f"  • Predicted sales change: {best['actual_effect']:+.1f}% (target: +20.0%)")
print(f"  • 90% Confidence Interval: [{best['ci_90'][0]:+.1f}%, {best['ci_90'][1]:+.1f}%]")
print(f"  • 50% Confidence Interval: [{best['ci_50'][0]:+.1f}%, {best['ci_50'][1]:+.1f}%]")
print(f"  • Confidence Score: {best['confidence']:.0%}")
print(f"  • Status: {'✅ APPROVED' if best['within_tolerance'] else '❌ NOT APPROVED'}")
print("="*70)

## 6. Compare Top Interventions

In [ ]:
# Show top 5 interventions
print("\nTop 5 Interventions:\n")
for i, candidate in enumerate(results['all_candidates'][:5], 1):
    print(f"{i}. {', '.join(candidate['nodes'])}")
    print(f"   Effect: {candidate['actual_effect']:+.1f}% | "
          f"Confidence: {candidate['confidence']:.0%} | "
          f"Quality: {candidate['quality']['overall_grade']}")
    print(f"   Changes: {candidate['required_pct_changes']}\n")

## 7. Path Analysis

Understanding which causal paths contribute most to the effect

In [ ]:
if 'path_analysis' in results:
    path_info = results['path_analysis']
    print("\nCausal Path Sensitivity Analysis:")
    print(f"  • Total paths analyzed: {path_info.get('total_paths', 'N/A')}")
    print(f"  • High quality paths: {path_info.get('high_quality_paths', 'N/A')}")
    print(f"  • Average path quality: {path_info.get('avg_path_quality', 0):.3f}")

## 8. Business Interpretation

### Key Insights:

1. **Primary Levers**: The analysis identifies which operational variables have the strongest causal impact on sales
2. **Confidence Levels**: High confidence scores indicate reliable predictions based on strong model quality
3. **Uncertainty**: Confidence intervals account for model uncertainty through Monte Carlo simulation
4. **Feasibility**: Interventions are validated for out-of-distribution detection and practical constraints

### Recommended Actions:

Based on the best intervention identified:
- Implement the recommended changes gradually
- Monitor actual vs. predicted outcomes
- Consider multi-node interventions for robust improvements
- Focus on high-quality causal paths for maximum reliability

## Summary

This notebook demonstrated:
- ✅ Loading and preparing retail data
- ✅ Defining causal graph structure
- ✅ Training causal models with HT
- ✅ Finding optimal interventions with uncertainty quantification
- ✅ Analyzing causal paths and model quality
- ✅ Interpreting results for business decisions